<a href="https://colab.research.google.com/github/VARUN-OFFICIAL-24/keyhunt/blob/main/plant_Doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 0) Installs
!pip -q install -U timm torchvision torchmetrics datasets tensorflow-datasets xgboost requests pyyaml

import os, json, math, time, random, yaml, io, requests, numpy as np
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple
from PIL import Image
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
import tensorflow_datasets as tfds
import xgboost as xgb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# Dataset Download Scripts for PlantAI Doctor
import os
import zipfile
import requests
import subprocess
from pathlib import Path
import tensorflow_datasets as tfds
from datasets import load_dataset
import shutil

# Create data directory
DATA_DIR = Path("/content/plant_datasets")
DATA_DIR.mkdir(exist_ok=True)

print("🌱 PlantAI Doctor - Dataset Downloader")
print("=" * 50)

# Configure which datasets to download
DOWNLOAD_PLANTVILLAGE = True
DOWNLOAD_PLANTDOC = True
DOWNLOAD_CASSAVA = True
DOWNLOAD_CCMT = True


🌱 PlantAI Doctor - Dataset Downloader


In [4]:
import tensorflow_datasets as tfds

(ds_train, ds_val, ds_test), info = tfds.load(
    "plant_village",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    as_supervised=True,
    with_info=True,
    download=True,                 # ensures generation
    data_dir="/content/tfds"       # optional: cache location
)

print("Classes:", len(info.features["label"].names))
print("Train examples:", info.splits["train"].num_examples)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /content/tfds/plant_village/incomplete.ZLRMSZ_1.0.2/plant_village-train.tfrecord*...:   0%|         …

Dataset plant_village downloaded and prepared to /content/tfds/plant_village/1.0.2. Subsequent calls will reuse this data.
Classes: 38
Train examples: 54303


In [6]:
builder = tfds.builder("plant_village", data_dir="/content/tfds")
builder.download_and_prepare()   # generates dataset under Drive
info = builder.info
train_ds = builder.as_dataset(split="train[:80%]", as_supervised=True)
val_ds   = builder.as_dataset(split="train[80%:90%]", as_supervised=True)
test_ds  = builder.as_dataset(split="train[90%:]", as_supervised=True)

In [8]:
import os
# Option A: per-call cache_dir
from datasets import load_dataset
pv = load_dataset("GVJahnavi/PlantVillage_dataset")
print(pv)

# Option B: global env (set before any HF calls)
# os.environ["HF_DATASETS_CACHE"] = f"{BASE}/hf_datasets"  # only Datasets cache
# or to move everything (datasets + hub assets):
# os.environ["HF_HOME"] = f"{BASE}/hf_home"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/320M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/361M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/171M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43503 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10878 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 43503
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10878
    })
})


In [19]:
from google.colab import files

# Upload your kaggle.json file
uploaded = files.upload()

# Move the uploaded file to the expected location
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  # Assuming the uploaded file is kaggle.json, move it to /content
  if fn == 'kaggle.json':
    import os
    os.rename(fn, '/content/kaggle.json')
    print("kaggle.json moved to /content/")
  else:
    print(f"Uploaded file {fn} is not kaggle.json. Please upload kaggle.json.")

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes
kaggle.json moved to /content/


In [20]:
import os
print(os.listdir('/content'))

['.config', 'plant_datasets', 'tfds', 'drive', 'kaggle.json', '.ipynb_checkpoints', 'kaggle', 'sample_data']


In [21]:
import os, zipfile, subprocess, pathlib
os.environ["KAGGLE_CONFIG_DIR"] = "/content"  # assumes kaggle.json uploaded to /content
OUT = "/content/kaggle"
pathlib.Path(OUT).mkdir(parents=True, exist_ok=True)

# Example: Cassava competition data into Drive
subprocess.run([
    "kaggle", "competitions", "download", "-c",
    "cassava-leaf-disease-classification", "-p", OUT
], check=True)

# Extract all zips into Drive
for z in pathlib.Path(OUT).glob("*.zip"):
    with zipfile.ZipFile(z, "r") as zip_ref:
        zip_ref.extractall(OUT)
    z.unlink()
print("Kaggle data stored in:", OUT)

CalledProcessError: Command '['kaggle', 'competitions', 'download', '-c', 'cassava-leaf-disease-classification', '-p', '/content/kaggle']' returned non-zero exit status 1.

In [22]:
# Install Kaggle CLI
!pip -q install kaggle

# Upload your kaggle.json (Profile → Account → Create New API Token) then run:
from google.colab import files
uploaded = files.upload()  # select kaggle.json

# Move into the default location and set perms (or keep KAGGLE_CONFIG_DIR=/content)
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Verify CLI works and is authenticated
!kaggle --version
!kaggle datasets list -s cassava | head


Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5
ref                                                                 title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
kingofarmy/cassavapreprocessed                                      cassava-preprocessed                                7173874538  2020-12-27 06:30:31.367000            592         44  0.7941176        
maksymshkliarevskyi/cassava-leaf-disease-models                     Cassava Leaf Disease Models                          135717252  2020-12-07 14:47:29.660000           1070         34  1.0              
nirmalsankalana/cassava-leaf-disease-classification                 Cassava Leaf Disease Classification                 2561676357 

In [24]:
import os, pathlib, zipfile, subprocess

# Optional: save to Drive
# from google.colab import drive
# drive.mount('/content/drive')
# OUT = "/content/drive/MyDrive/plant_ai_data/kaggle"  # persistent

# Public datasets endpoint (no competition rules required)
OUT = "/content/kaggle"
pathlib.Path(OUT).mkdir(parents=True, exist_ok=True)

res = subprocess.run(
    ["kaggle", "datasets", "download", "-d", "nirmalsankalana/cassava-leaf-disease-classification", "-p", OUT],
    capture_output=True, text=True
)
print("stdout:", res.stdout)
print("stderr:", res.stderr)
res.check_returncode()

for z in pathlib.Path(OUT).glob("*.zip"):
    with zipfile.ZipFile(z, "r") as zip_ref:
        zip_ref.extractall(OUT)
    z.unlink()
print("Cassava dataset (datasets endpoint) stored in:", OUT)



stdout: Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /content/kaggle.json'
Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/cassava-leaf-disease-classification
License(s): CC0-1.0


stderr: 
  0%|          | 0.00/2.39G [00:00<?, ?B/s]
  4%|▍         | 109M/2.39G [00:00<00:02, 1.08GB/s]
  9%|▊         | 212M/2.39G [00:00<00:03, 718MB/s] 
 12%|█▏        | 287M/2.39G [00:00<00:05, 381MB/s]
 14%|█▍        | 337M/2.39G [00:00<00:05, 371MB/s]
 16%|█▌        | 381M/2.39G [00:02<00:29, 73.9MB/s]
 19%|█▊        | 457M/2.39G [00:03<00:18, 113MB/s] 
 21%|██        | 515M/2.39G [00:03<00:13, 148MB/s]
 23%|██▎       | 564M/2.39G [00:04<00:27, 71.1MB/s]
 25%|██▍       | 599M/2.39G [00:07<00:44, 43.1MB/s]
 28%|██▊       | 682M/2.39G [00:07<00:25, 71.3MB/s]
 30%|███       | 734M/2.39G [00:07<00:19, 93.3MB/s]
 32%|███▏      | 779M/2.39G [00:11<00:50, 34.8MB/s]
 35%|███▌      | 862M/2.39G [00:11<00:29, 56.2MB/s]
 37%|███▋      | 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, shutil, pathlib
SRC = pathlib.Path("/content/kaggle")
DST = pathlib.Path("/content/drive/MyDrive/plant_ai_data/cassava")
DST.mkdir(parents=True, exist_ok=True)

# Copy (safer than move across devices)
def copytree(src, dst):
    for p in src.rglob("*"):
        rel = p.relative_to(src)
        tgt = dst / rel
        if p.is_dir():
            tgt.mkdir(parents=True, exist_ok=True)
        else:
            tgt.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(p, tgt)

copytree(SRC, DST)
print("Copied to Drive:", DST)

# Optional: check a few files
for i, p in enumerate(DST.rglob("*")):
    if i > 10: break
    print(p)


Mounted at /content/drive


In [ ]:
import pandas as pd
from pathlib import Path

ROOT = DST  # path under Drive
all_imgs = list(ROOT.rglob("*.jpg")) + list(ROOT.rglob("*.png")) + list(ROOT.rglob("*.jpeg"))
print("Found images:", len(all_imgs))

csvs = list(ROOT.rglob("*.csv"))
print("Found CSVs:", [c.name for c in csvs])

# Try to identify a primary CSV and likely columns
df = None
for c in csvs:
    try:
        tmp = pd.read_csv(c)
        print(c.name, "columns:", list(tmp.columns)[:10])
        if ("label" in tmp.columns and ("image" in tmp.columns or "image_id" in tmp.columns or "filename" in tmp.columns)):
            df = tmp
            csv_path = c
            break
    except Exception as e:
        pass

if df is not None:
    print("Using CSV:", csv_path.name, "rows:", len(df))
else:
    print("No suitable CSV found; will use folder names as class labels if available.")
